<a href="https://colab.research.google.com/github/Satorumi/Machine-Learning/blob/main/OpenCV_Projects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Attendance with Face Recognition Project**

In [2]:
!pip install face-recognition

     |████████████████████████████████| 100.2MB 91kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566184 sha256=67d0509c042a25d0aca0666b47505998e0728f84a59c64bf91bfdee5b968609d
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [47]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Harry Styles.jpeg to Harry Styles.jpeg
Saving Taylor Swift.jpg to Taylor Swift (1).jpg
User uploaded file "Harry Styles.jpeg" with length 80962 bytes
User uploaded file "Taylor Swift.jpg" with length 739889 bytes


In [3]:
import cv2 as cv
from google.colab.patches import cv2_imshow
import numpy as np
import dlib
import face_recognition as fr
import os
from datetime import datetime
import csv

#####Using face_recognition

In [37]:
# import train and test image
el_train = fr.load_image_file('el_train.jpg')
el_test = fr.load_image_file('el_test.jpg')

In [39]:
# convert color because the model only take in RGB color img
rgb_train = cv.cvtColor(el_train, cv.COLOR_BGR2RGB)
rgb_test = cv.cvtColor(el_test, cv.COLOR_BGR2RGB)

In [41]:
face_location = fr.face_locations(rgb_train)[0] # define face location
w, x, y, h = face_loc
encoded_train = fr.face_encodings(rgb_train)[0]
encoded_test = fr.face_encodings(rgb_test)[0]
  # draw the face location
cv.rectangle(el_train, pt1=(x,y), pt2=(w, h), color=(0,0,255), thickness=2)
cv2_imshow(el_train)
face_location

(139, 324, 325, 139)

In [42]:
# check if it recognized as the same person
compare = fr.compare_faces([encoded_train], encoded_test) 
# compute distance between 2 img, the smaller distance, the more similar
face_dis = fr.face_distance([encoded_train], encoded_test)
compare, face_dis

([True], array([0.2662764]))

In [ ]:
# put text
cv.putText(el_train, (x+6, y-6), f'Comparision: {compare} / Face Distance{face_dis}',\
           cv.FONT_HERSHEY_COMPLEX, 1, (0,0,255), thickness=1)

###Attendance taking

In [ ]:
# get all imgs from loal file
path = 'C:\Users\lisa0\OneDrive\Images'
img_paths = os.listdir(path)
attendances = []
images = []
for img_path in img_paths:
  img = cv.imread(f'{path}/{img_path}')
  images.append(img)
  attendances.append(os.path.splitext(cl)[0]) # get the name of the person

In [ ]:
# define a func to encode imgs
def encodding_img(imgs): # take in a list of img
  encoded_imgs = []
  for img in imgs:
    rgb_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)   
    encoded_img = fr.face_encodings(rgb_img)[0]
    encoded_imgs.append(encoded_img)
  return encoded_imgs

encoded_imgs = econdding_img(images) # pass in img list

In [ ]:
cap = cv.VideoCapture(0)

while True:
  _, frame = cap.read()
  frameS = cv.resize(frame, (0,0), None, 0.25, 0.25) # scale down img
  frameS = cv.cvtColor(frame, cv.COLOR_BGR2RGB) 
  # might be multiple faces
  face_locations = fr.face_locations(frameS)[0] # define face location
  w, x, y, h = face_locations * 4 # orginal location
  encoded_face = fr.face_encodings(frameS)[0] # encode face

  # loop through each face detected
  for face, loc in zip(encoded_face, face_locations):
    match = fr.compare_faces(encoded_imgs, face) # compare it with previous faces
    dist = fr.face_distance(encoded_imgs, face) # compute distances
    most_similar = attendances[np.argmin(dist)] # take the least dist and get the name
    if match[np.argmin(dist)]: # check if match
      # draw rectangle aroung detected face
      cv.rectangle(el_train, pt1=(x,y), pt2=(w, h), color=(0,0,255), thickness=2)
      # write the name of matched person 
      cv.putText(frameS, (x+6, y-6), most_similar,
           cv.FONT_HERSHEY_COMPLEX, 1, (0,0,255), thickness=1)
      
      mark_attendance(most_similar) # use a function to mark attendance into csv file
      
  cv2_imshow(frameS)
    

In [ ]:
def mark_attendance(name):
    # using panda to write in csv
  # attendances_info = pd.read_csv('attendance.csv')
  # if name not in attendances_info['name']:
  #   time = datetime.now().strftime('%H-%S-%S %Y-%m-%d')
  #   attendances_info.append({'name': name, 'attendance': time})
  # attendances_info.to_csv('attendance.csv')

  # Using csv library
  with open('attendance.csv', 'r+') as f:
    attendances_info = f.readlines()
    attended = [] # already attended people
    for info in attendances_info:
      entry = info.split(',')
      attended.append = entry[0]
    if name not in attended:
      time = datetime.now().strftime('%H-%S-%S %Y-%m-%d') # current time
      writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      writer.writerow([name, time]) # write new row


## **QR-CODE AND BARCODE DETECTOR**

In [2]:
import numpy as np
from pyzbar.pyzbar import decode
import cv2 as cv

import imutils

In [ ]:
# set up camera
cap = cv.VideoCapture(0)
camera_width, camera_height = 640, 480
cap.set(3, camera_width)
cap.set(4, camera_height)

In [ ]:
while True:
  _, frame = cap.read()
  barcodes = decode(frame) # get the barcode from img
  for barcode in barcodes: # in case of multiple barcodes
    # barcode data is a bytes object so we have to decode it to string
    data = barcode.data.decode('utf-8')

    # check if data is authorized
    with open('authorized_data.csv', 'wb') as f: 
      authorized_data = f.read()
      if data in authorized_data:
        color = (0,255,0)
      else:
        color = (0,0,255)
    # get the polygon shape of barcode
    pts = np.array([barcode.polygon], np.int32).reshape((-1,1,2)) # reshape the array
    data.polylines(frame, [pts], True, color, 5) # boundng polygon
    x, y, w, h = barcode.rect # get the point of rectangle shape of qr-code
    # write received data
    cv.putText(frame, (x, y), data, cv.FONT_HERSHEY_COMPLEX, 1, color), 2)
  cv.imshow(frame)

####BarCode detector
refs: 
- https://www.pyimagesearch.com/2014/11/24/detecting-barcodes-images-python-opencv/
- https://laconicml.com/barcode-detection-opencv/


In [ ]:
cap = cv.VideoCapture(0)

## Process the image to detect barcode
while True:
  _, img = cap.read()
  gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

  # find the gradient of the image with sobel
  depth = cv.cv.CV_32F if imutils.is_cv2() else cv.CV_32F
  xgrad = cv.sobel(gray_img, depth, dx=1, dy=0, ksize=-1)
  ygrad = cv.sobel(gray_img, depth, dx=0, dy=1, ksize=-1)
  # subtracting two grad
  grad_img = cv.subtract(xgrad, ygrad) # result in high hor, low ver gradients
  #Scales, calculates absolute values
  grad_img = cv.convertScaleAbs(grad)

  # blurr to smoothen the noise
  blurred = cv.blurr(grad_img, ksize =(9,9))
  # pixel in gradient > 255 -> 255, otherwise black (0)
  _, thresh = cv.threshold(blurred, 255, 255, cv.THRESH_BINARY)

  # apply a close kernel to the threshold img
  # construct a closed kernel
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
  closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
  # erod and dilation to remove excess white area 
  # and close the black space arounf bar code
  closed = cv2.erode(closed, None, iterations = 4)
  closed = cv2.dilate(closed, None, iterations = 4)

  # find contours and draw it
  contours, _ = cv.findcountours(closed.copy()), cv.RETR_EXTERNAL, CV.CHAIN_APPROX_SIMPLE)
  contours = imutils.grab_contours(contours)
  # get the max area contour
  max_c = sorted(contours, key=cv.contourArea, reverse=True)[0]

  # calculate the area of bounding box
  bbox = cv.minAreaRect(max_c)
  # the location of bbox
  bbox = cv.cv.BoxPoints(bbox) if imutils.is_cv2() else cv2.boxPoints(bbox)
  bbox = np.int0(box) # numpy method to convert bbox float point to int

  # draw the contour on the img
  cv.drawContour(img, bbox, -1, (0,0,255, 3))

  cv2.imshow("Image", img)
  cv2.waitKey(0)